In [ ]:
pip install stanza

In [ ]:
################################################## IMPORTACIONES #######################################################
import os

import json

# LIBRERIA PARA STOPWORDS Y TOKENIZER
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# LIBRERIA PARA ANALISIS LE LENGUAJE NATURAL DESARROLLADA CON REDES NEURONALES
import stanza

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# INSTALANDO DEPENDENCIAS PARA PODER OPERAR CON EL LENGUAJE ESPAÑOL
stanza.download('es', package='ancora', processors='tokenize,mwt,pos,lemma', verbose=True)
stNLP = stanza.Pipeline(processors='tokenize,mwt,pos,lemma', lang='es', use_gpu=True)

2021-09-22 21:11:18 INFO: Downloading these customized packages for language: es (Spanish)...
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| pretrain  | ancora  |

2021-09-22 21:11:18 INFO: File exists: /root/stanza_resources/es/tokenize/ancora.pt.
2021-09-22 21:11:18 INFO: File exists: /root/stanza_resources/es/mwt/ancora.pt.


2021-09-22 21:11:36 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-09-22 21:11:36 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2021-09-22 21:11:36 INFO: Use device: cpu
2021-09-22 21:11:36 INFO: Loading: tokenize
2021-09-22 21:11:36 INFO: Loading: mwt
2021-09-22 21:11:36 INFO: Loading: pos
2021-09-22 21:11:37 INFO: Loading: lemma
2021-09-22 21:11:37 INFO: Done loading processors!


In [ ]:
############################################### FUNCIONES ADICIONALES ##################################################

# FUNCION QUE AYUDA A PREVENIR LA MALA IDENTIFICACION DEL PESO, CON EL 'TITULO' Y 'OBJETO DE LEY'
def UnirVariables(array1, array2):
    array=[]
    for word in array2:
        if (i not in array1):
            array.append(word)
    return array

################################################# LIMPIEZA DE RUIDO ####################################################

def Lematizar(word):
    # LEMATIZAMOS EL PARAMETRO (PALABRA)
    doc = stNLP(word)
    # RECUPERAMOS EL LEMA DEL WORD
    aux = doc.sentences[0].words[0]
    return (aux.lemma)

def Preproccessing_Personalizado(filtered):

    important_word = ["ley", "legislativo", "supremo", "urgencia"]
    final = []
    controlador = False
    word = ""
    # ARREGLO PERSONALIZADO LEYES
    for i in range(0, len(filtered)):
        if controlador == False:
            if (filtered[i] in important_word):
                if (filtered[i] == "legislativo" or filtered[i] == "supremo" or "urgencia"):
                    if (filtered[i - 1] == "resolución"):
                        word = "resolución legislativa"
                    if (filtered[i - 1] == "decreto"):
                        word = "decreto " + filtered[i]
                    else:
                        word = filtered[i]
                controlador = True
            else:
                final.append(filtered[i])

        else:
            if (any(chr.isdigit() for chr in filtered[i])):
                new_word = word + "(" + filtered[i] + ")"
                final.append(new_word)
                final.append(new_word)
                final.append(new_word)
                controlador = False
            else:
                controlador = False
    return final

def Preprocessing(texto):
    # TOKENIZAMOS EL TEXTO
    word_tokens = word_tokenize(texto)

    # RECUPERAMOS LOS STOP_WORDS DE LA LIBRERIA NLTK.CORPUS
    stop_words = set(stopwords.words('spanish'))

    # NUESTROS STOPWORDS
    our_stopwords = {"propone", "así","uso","u","art","numeral","general","regl","prov","núm","mediante",
                     "comision", "titulo","pasa","artítulo","artitulo","oficio","pleno","texto","aras",
                     "aquella", "tiene", "expresamente", "pequeña","ser","y/o","cada","parte", "lay",
                     "nacional","interés","interes","público","publico","constitución", "declarar","necesidad",
                     "perú","proceso","disposición","forma","año","bien","mínimo","mínimo","alto","artículo","articulo",
                     "provincia","región", "objeto", "nuevo","permitir","vida","centro","función","funcion",
                     "objeto","medida","caso","peruano","departamento","san","ubicado","regional","nivel","marco",
                     "actividad","país","día","inciso","valor","plazo","red","básico","literal","libre","medio","dentro",
                     "santo","código","vía","río","recurso","villa","miembro","área","primero","segundo","tercero",
                     "zona","distrito","pago","crisis","fin","post"}

    # UNIMOS LOS STOPWORDS DE LA LIBRERIA CON NUESTROS STOPWORDS
    stop_words = stop_words.union(our_stopwords)

    # FILTRAMOS EL ARREGLO TOKENIZADO
    filtered = []
    for i in range(1, len(word_tokens)):
        # CONVERTIMOS A MINUSCULA
        word_tokens[i] = word_tokens[i].lower()

        # ELIMINAMOS NÚMEROS
        # word_tokens[i] = re.sub("\d+", ' ',word_tokens[i])

        # ELIMINAMOS SIGNOS DE PUNTUACION
        signos = ["?", "¿", "¡", "!", " ", ",", ".", ";", ":"]
        if (word_tokens[i] in signos): word_tokens[i] = ' '

        # ELIMINAMOS ESPACIOS
        word_tokens[i] = re.sub('\s+', ' ',word_tokens[i])

        # VALIDAMOS QUE SEAN DIFERENTES A VACIO POR LA FUNCION R.SUB
        if ( word_tokens[i] != ' ' ):
            # ELIMINAMOS LOS STOPWORDS
            if ( word_tokens[i] not in stop_words):

                # REALIZAMOS LA LEMATIZACION
                word_tokens[i] = Lematizar(word_tokens[i])
                if (word_tokens[i] not in stop_words):

                    # VALIDAMOS QUE SEAN MAYORES A UNO YA QUE SE NOMBRAN ARTICULOS '4-A, 4-B'
                    if (len(str (word_tokens[i]))>2):
                        # AGREGAMOS AL VECTOR FINAL PREPROCESADO
                        filtered.append(word_tokens[i])

    # PREPROCESADO PERSONALIZADO (PARA VISUALIZAR LEYES MODIFICADAS)
    final_filtered = Preproccessing_Personalizado(filtered)

    # RETORNAMOS EL ARREGLO PREPROCESADO
    return final_filtered

In [ ]:

################################################## RECORRER JSONS ######################################################

# LISTAR DIRECTORIOS DE NUESTRO CORPUS
url_JSONs = os.listdir('./CORPUS-JSON')

In [ ]:
url_JSONs

['2019.json', '2016.json', '2017.json', '2018.json', '2020.json', '2021.json']

In [ ]:
#### CREAMOS EL JSON (LEY <==> ARREGLO DE PALABRAS PREPROCESADAS)
data1 = {}
data1['leyes'] = []

# RECORRER ARCHIVOS JSON RECUPERADOS DEL CORPUS
for i in url_JSONs:
  print(i)
  # ABRIR ARCHIVO JSON (TENER MUY EN CUENTA EL ENCODING)
  with open('./CORPUS-JSON/'+i, encoding="utf-8") as file:
    data = json.load(file)
    # RECUPERAMOS ATRIBUTOS IMPORTANTES DEL JSON '(TITULO, OBJETO DEL PROYECTO)'
    for j in data:
        for ley in data[j]:
          titulo = ley['titulo']
          objeto_proyecto = ley['objeto_del_proyecto']
           ### REALIZAMOS EL PREPROCESADO DE TITULO Y OBJETO DE LEY
          pre_titulo = Preprocessing( titulo )
          pre_objeto = Preprocessing( objeto_proyecto )
          # UNIR AMBOS PREPROCESADOS YA QUE EXISTEN TERMINOS QUE SE REPITEN EN TITULO Y OBJETO DEL PROYECTO
          preprocesado = UnirVariables( pre_titulo, pre_objeto )
          # AGREGAMOS AL ARCHIVO JSON (LEY <==> ARREGLO DE PALABRAS PREPROCESADAS)
          data1['leyes'].append({
              'ley': str(ley["numero_ley"].replace(".json", "")),
              'words': preprocesado
          })


2019.json


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


2016.json
2017.json
2018.json
2020.json
2021.json


In [ ]:
# GUARDAMOS EL JSON (LEY <==> ARREGLO DE PALABRAS PREPROCESADAS)
nombre_archivo = "./PREPROCESADO-LEYES.json"
with open(nombre_archivo, 'w', encoding="utf-8") as file:
    json.dump(data1, file, indent=4, ensure_ascii=False)

In [ ]:
pip install stylecloud

     |████████████████████████████████| 262 kB 7.4 MB/s 
     |████████████████████████████████| 161 kB 66.8 MB/s 
     |████████████████████████████████| 87 kB 8.6 MB/s 
     |████████████████████████████████| 87 kB 9.2 MB/s 
  Created wheel for stylecloud: filename=stylecloud-0.5.2-py3-none-any.whl size=259508 sha256=7f7921b23006e2dc4cc5de5f603e953c3f62b145117ce501565c616f2baf3cc7
  Stored in directory: /root/.cache/pip/wheels/77/42/7a/f52b5f84c04196fd4c2a3dceeb1bbeaee1c93a4fe271b5eb41
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=d9e6f3e5c119d2a739f701a5f38014e383d5a13b25a702bb6d2e4e1741cd9667
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for tinycss: filename=tinycss-0.4-py3-none-any.whl size=43955 sha256=eeb9cea07169e08fcb31d8eaef11f0a3bb2bd00288357d26a96a18d4f33a50a0
  Stored in directory: /root/.cache/pip/wheels/6c/66/e8/e53d7a476011891fa51a5ee83a2d1852b19b258f975

In [ ]:
pip install stylecloud

In [ ]:
################################################## IMPORTACIONES #######################################################
import json
import openpyxl
# LIBRERIA PARA REALIZAR LA MATRIZ DE BAG OF WORDS
from keras.preprocessing.text import Tokenizer

# LIBRERIA PARA REALIZAR NUBES DE PALABRAS
import stylecloud as sc

In [ ]:
############################################## GENERAR BAG OF WORDS ####################################################

# RECUPERAR EL ARREGLO DE WORDS GENERAL
array_words = []
nombre_leyes = []

# RECUPERAMOS LA URL DEL JSON
url_creator = "./PREPROCESADO-LEYES.json"

# ABRIMOS EL ARCHIVO JSON
with open(url_creator, encoding="utf-8") as file:
    data = json.load(file)

    # RECORREMOS EL JSON Y RECUPERAMOS EL ARREGLO DE WORDS Y EL IDENTIFICADOR DE LA LEY
    for aux in data['leyes']:
        array_words.append( aux['words'] )
        nombre_leyes.append( aux['ley'] )

# REALIZAMOS EL FIT DE LAS PALABRAS DEL ARREGLO PREPROCESADO
model = Tokenizer()
model.fit_on_texts(array_words)

# ARREGLO DE PALABRAS PREPROCESADAS
keys = list(model.word_index.keys())

# REALIZAMOS LA MATRIZ DE BAG OF WORDS
rep = model.texts_to_matrix(array_words, mode='count')

In [ ]:
################################################ CONSTRUIMOS EXCEL #####################################################

# HABILITAMOS LA LIBRERIA QUE NOS PERMITE TRABAJAR CON UN ARCHIVO EXCEL
wb = openpyxl.Workbook()
hoja = wb.active

# GENERAMOS LAS FILAS USANDO LA MATRIZ DE BAG OF WORDS
filas = [ ]
for i in range (len(nombre_leyes)):
    aux = list(rep[i])
    del aux[0]
    fila = [nombre_leyes[i]] + aux
    filas.append(fila)

# CREAMOS LA FILA CON LOS ENCABEZADOS (ARRAY_WORDS)
array = [''] + keys
hoja.append(array)




# AGREGAMOS LAS FILAS A NUESTRA HOJA DE EXCEL
for producto in filas:
    # producto es una tupla con los valores de un producto
    hoja.append(producto)
    #print( producto[0] )

wb.save('BOW2.xlsx')


In [ ]:
print(keys)

['transparencia', 'modificar', 'gobierno', 'información', 'decreto', 'decreto legislativo(1275)', 'establecer', 'fiscal', 'responsabilidad', 'acceso', 'garantizar', 'local', 'tribunal', 'constitucional', 'privado', 'fortalecer', 'ley(28301)', 'ley(27806)', 'principio', 'protección', 'administración', 'política', 'institución', 'incorporar', 'poder', 'complementario', 'corrupción', 'contratación', 'publicidad', 'transparentar', 'república', 'aprobar', 'ley(30356)', 'decreto legislativo(1071)', 'dato', 'personal', 'autoridad', 'sector', 'ley(30099)', 'financiero', 'reglamento', 'congreso', 'procedimiento', 'organización', 'ejecución', 'entidad', 'materia', 'empresa', 'lucha', 'buen', 'derecho', 'elección', 'regular', 'deber', 'organismo', 'obra', 'internacional', 'arbitraje', 'gestión', 'cumplimiento', 'control', 'final', 'decreto legislativo(1508)', 'sistema', 'través', 'regulación', 'consejo', 'maximizar', 'criterio', 'eficiencia', 'comisión', 'judicial', 'democracia', 'probidad', 'cos

In [ ]:
print(filas)

[['06807/2020-PE', 1.0, 2.0, 3.0, 0.0, 4.0, 3.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1

In [ ]:
print(rep)

[[0. 1. 2. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 3. 0. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 1. 1.]]


In [ ]:
################################################ NUBE DE PALABRAS ######################################################

'''
# GENERAMOS EL TEXTO, USANDO LOS ARRAY_WORDS
texto = ""
for j in array_words:
    for i in range (len(j)):
        texto += j[i] + " "
'''

# DICCIONARIO PARA LA NUBE DE PALABRAS
dic_ponderado = {}

lista = list(model.word_docs.items())
print(lista)
for i in range(len(lista)):
    dic_ponderado[str(lista[i][0])] = lista[i][1]

# ORDENAMOS DICCIONARIO DE PALABRAS PREPROCESADAS PARA PASARLO A UN EXCEL
sorted_words = sorted(dic_ponderado.items(), key=lambda x: x[1], reverse=True)
wb1 = openpyxl.Workbook()
hoja1 = wb1.active
for i in range(len(sorted_words)):
    fila = [ sorted_words[i][0], sorted_words[i][1]]
    hoja1.append(fila)
url_save = './BOWSuma.xlsx'
wb1.save(url_save)

# NUMERO DE PALABRAS
print(len(keys))

# CREAMOS LA NUBE DE PALABRAS Y LA GUARDAMOS
url_save_png = './NUBE DE PALABRAS-TF.png'

sc.gen_stylecloud(
    text= dic_ponderado,
    colors=['#ecf0f1', '#3498db', '#e74c3c'],
    icon_name='fas fa-cloud',
    background_color='#1A1A1A',
    output_name = url_save_png,
    size = 2048,
    max_words = 300,
)

[('oportuno', 2), ('régimen', 3), ('mismo', 2), ('supervisión', 2), ('efecto', 3), ('públicas', 3), ('garantizar', 12), ('efectivo', 3), ('establecer', 13), ('responsabilidad', 13), ('profesional', 3), ('transparentar', 6), ('ejecución', 5), ('acreditación', 1), ('igual', 1), ('ejercer', 2), ('obra', 4), ('satisfacción', 1), ('especial', 2), ('mejor', 2), ('finalidad', 3), ('anuncio', 1), ('realizar', 1), ('transparencia', 46), ('contrato', 2), ('prensa', 2), ('materia', 3), ('conocimiento', 1), ('comunicación', 2), ('costo', 3), ('dimensión', 1), ('publicidad', 5), ('gasto', 3), ('televisivo', 1), ('real', 1), ('publicitario', 1), ('ciudadano', 3), ('escrito', 1), ('favorecer', 1), ('privado', 7), ('radial', 1), ('contratar', 1), ('publicar', 1), ('utilización', 2), ('rendición', 1), ('extractiva', 1), ('iniciativa', 1), ('implementación', 1), ('obtenido', 1), ('sector', 5), ('gobierno', 14), ('promover', 3), ('cuenta', 1), ('hidrocarburo', 1), ('empresa', 3), ('criterio', 3), ('estab

In [ ]:
sc.gen_stylecloud(
    text= dic_ponderado,
    colors=['#ecf0f1', '#3498db', '#e74c3c'],
    icon_name='fas fa-cloud',
    background_color='#1A1A1A',
    size = 2048,
    max_words = 300,
)